In [0]:
import pandas as pd
import scipy.stats as st
from statsmodels.stats.multitest import multipletests

In [0]:
data = pd.read_csv('classifiers.csv')

In [3]:
data

,Номер выборки,a1,a2,a3,a4
0,1,86,50,93,13
1,2,85,74,55,35
2,3,53,92,58,51
3,4,44,41,56,37
4,5,2,18,99,26
5,6,5,68,35,17


In [4]:
data.describe()

,Номер выборки,a1,a2,a3,a4
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,3.500000,45.833333,57.166667,66.000000,29.833333
std,1.870829,36.853313,26.309061,24.754798,14.062954
min,1.000000,2.000000,18.000000,35.000000,13.000000
25%,2.250000,14.750000,43.250000,55.250000,19.250000
50%,3.500000,48.500000,59.000000,57.000000,30.500000
75%,4.750000,77.000000,72.500000,84.250000,36.500000
max,6.000000,86.000000,92.000000,99.000000,51.000000


In [5]:
# Посмотрим, можно ли отклонить какую-нибудь гипотезу используя критерий Уилкоксона без поправки на множественную проверку
for i in range(1, 5):
  for j in range(i+1, 5):
    _, p_val = st.wilcoxon(data.iloc[:, i].values, data.iloc[:, j].values)
    print(data.columns[i], data.columns[j], p_val)

a1 a2 0.463071015014588
a1 a3 0.2071604489114608
a1 a4 0.463071015014588
a2 a3 0.7531523647659145
a2 a4 0.07473549830588248
a3 a4 0.027707849358079864


/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


В этом случае гипотеза о неразличимости моделей может быть отклонена только для a3 и a4: $$p-value(a3, a4) < \alpha = 0.05$$

Сделаем попровку на множественость гипотез, воспользуемся критерием Тьюки:

In [0]:
data_numpy = data.to_numpy()[:, 1:]
n, k = data_numpy.shape
s = data_numpy.std() * (n / (n - 1))**(1/2) # множитель нужен, т.к. data_numpy.std() = sqrt(mean(abs(x - x.mean())**2)
# s = data_numpy.std()

In [0]:
def Tukey(x, y):
  return abs(x.mean() - y.mean()) / (s * (n/2)**(1/2))

In [8]:
for i in range(1, 5):
  for j in range(i+1, 5):
    print(data.columns[i], data.columns[j], 
          Tukey(data.iloc[:, i], data.iloc[:, j]))

a1 a2 0.21404064356929542
a1 a3 0.38086643929242286
a1 a4 0.3021750262154761
a2 a3 0.16682579572312742
a2 a4 0.5162156697847715
a3 a4 0.683041465507899


Критическое значение статистики для $k=4, n=6, \alpha=0.05 \rightarrow m=2.9$

Все значения статистик меньше этого значения, ни одну из гипотез отклонить не можем.